!["question"](question.jpg)



between layer 1 and 2:

$$
\begin{bmatrix}
  b_{10} & w_{11} & w_{21} \\
  b_{20} & w_{12} & w_{22} 
\end{bmatrix}
$$

where b = bias and w = weights

$x = [0.1,0.2]$

In [44]:
import numpy as np
from random import seed

input is columns, output is rows 

In [100]:
def Linear(num_nodes_input_layer, num_nodes_output_layer):
    """Dimensions: output is rows, input is columns."""
    seed(1)
    weights = np.random.rand(num_nodes_output_layer, num_nodes_input_layer + 1)
    # biases = np.ones((weights.shape[0], 1))

    # Stack the original array and the column of ones
    # return np.column_stack((biases, weights ))
    return weights


# Forward propogation to make predictions

!["how to apply the weights"](apply_weights.png)

$$ output = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [392]:
def compute_forward(inputs, layer):
    ## linear combination w0x0 + w1x1 + w2x2
    linear_combinations = []
    # print(inputs, inputs.shape)
    # print(np.ones(inputs.shape[1]))
    inputs = np.c_[ np.ones(inputs.shape[0]), inputs  ]    ## add bias to the inputs
    # print(inputs)
    # print(layer)

    linear_combinations = np.matmul(inputs, layer.T) ## logits or z
    
    # print(linear_combinations)

    sigmoids = 1/(  1 + np.exp( -linear_combinations ) ) ## compute a
    # print(sigmoids)

    return linear_combinations, sigmoids


# def sigmoid(inputs, layer):

#     linear_combinations = np.matmul(inputs, layer.T) ## logits
#     return 1/(  1 + np.exp( -linear_combinations ) ) ## compute a

# Loss function

*m is number of samples*

*k is total number of output units*
$$
 J(θ)= - \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(\frac{1}{1+e^{xθ^T}})_k + (1-y^{(i)}_k)ln(1-(\frac{1}{1+e^{xθ^T}})_k)]} 
$$
$$
 = \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(a^{(L)}_k + (1-y^{(i)}_k)ln(1-a^{(L)}_k)]}

$$


$$  
output\_sigmoids = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [399]:
def binary_cross_entropy_loss(output_sigmoids, target):
    
    ## m is number of samples
    ## k is total number of output units

    log_sigmoids = np.log(output_sigmoids)
    y_times_log_sigmoids = np.matmul(log_sigmoids, target.T)
    # print(y_times_log_sigmoids)

    one_minus_log_sigmoids = 1 - np.log(output_sigmoids)
    one_minus_y_times_log_sigmoids = np.matmul(one_minus_log_sigmoids, (1 - target).T)
    # print(one_minus_y_times_log_sigmoids)
    
    return sum(sum((y_times_log_sigmoids + one_minus_y_times_log_sigmoids)))/len(target) ## sum across the output units and sum across the samples
    # for sample in target:
    #     for row in output_sigmoids:
            
    #         print(row, sample)
    #     print("_____________")




# Backpropogation

want: $\frac{∂J(θ)}{∂θ^{(l)}_{ij}}$ for $l=1,2$

for each

$$δ^{(L)} = \begin{bmatrix}
  a^{(L)}_{1} - y_{11} \\
  a^{(L)}_{2} - y_{12} \\
  a^{(L)}_1 - y_{21} \\ 
  a^{(L)}_1 - y_{22} \\ 

\end{bmatrix}
$$

$y_{11}$ is sample $1$, for target $1 $

$$δ^{(L)} * \begin{bmatrix} a^{(L-1)}_{0}  a^{(L-1)}_{1} a^{(L-1)}_2 \end{bmatrix} = \begin{bmatrix}
  \frac{∂J(θ)}{∂θ^{(L-1)}_{10}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{11}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{12}} \\
  \\
  \frac{∂J(θ)}{∂θ^{(L-1)}_{20}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{21}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{22}} \\


\end{bmatrix} $$

slides 37, 50

In [380]:
x = np.array([[0.1,0.2],
            [0.3,0.8], 
            [0.3,0.8]]) # x1, x2

y = np.array([[1,0],[1,1],[0,1]])

In [381]:
## define the layers
# seed(1)
#input = Linear(3,2)
seed(1)
hidden_layer1 = Linear(2,2)
seed(1)
output_layer = Linear(2,2)

# hidden_layer1

In [393]:
# sigmoids2, a2 = compute_forward(x, input)

# sigmoids3, a3 = compute_forward(a2, hidden_layer1)

z2, sigmoids2 = compute_forward(x, hidden_layer1)

z_output, output_sigmoids = compute_forward(z2, output_layer)


In [401]:
binary_cross_entropy_loss(output_sigmoids, y)

1.6995779071347037

In [485]:
def compute_backpropogation(sigmoids: list, layers: list, y, x):
    """"""
    # layers = [hidden_layer1, output_layer]
    layers.reverse()
    # sigmoids = [sigmoids2, output_sigmoids]
    sigmoids.insert(0, x)
    sigmoids.reverse()
    gradients = []


    for i, layer in enumerate(layers):
        if i == 0:
            delta = (sigmoids[i] - y)
            # print(delta.shape)
            previous_sigmoids = np.c_[ np.ones(sigmoids[i+1].shape[0]), sigmoids[i+1]  ]    ## add bias to the inputs
            # print(sigmoids[i])
            # print("__ __ __ __")
            # print(previous_sigmoids)

            ## delta has shape (samples x nodes)
            ## sigmoids has shape (samples x activations + 1)
            ## ex delta = sx2 (s = samples & nodes = 2 )
            ##    sigmoids = sx3 (s = samples & there are 2 sigmoids and 1 to multiple the bias by)
            gradients.append(np.matmul(delta.T, previous_sigmoids))


        else:
            # print("_______________________________")
            ## after the first delta, every previous delta is
            ## (delta * (weights from the previous layer)) * sigmoids or x
            delta = np.matmul(delta, layers[i-1][:,1:]) * sigmoids[i] * (1- sigmoids[i])
            previous_sigmoids = np.c_[ np.ones(sigmoids[i+1].shape[0]), sigmoids[i+1]  ]    ## add bias to the inputs
            # print(previous_sigmoids)
            gradients.append(np.matmul(delta.T, previous_sigmoids))


    # print(gradients)
    return gradients

def step(weights: list, gradients: list, lr = 0.001):
    gradients.reverse()
    for weight, gradient in zip(weights, gradients): ## so the first gradient is the input layers graidents
        yield weight - (gradient * lr)

    


In [506]:
### put it all together:
EPOCHS = 20
x = np.array([[0.1,0.2],
            [0.3,0.8], 
            [0.3,0.8]]) # x1, x2

# y = np.array([[1,0],[1,1],[0,1]])
y = np.array([[0,0],[0,1],[0,0]])


seed(1)
hidden_layer1 = Linear(2,2) 
seed(1)
output_layer = Linear(2,2) 

print(hidden_layer1, output_layer)
for epoch in range(EPOCHS):

    z2, sigmoids2 = compute_forward(x, hidden_layer1)

    z_output, output_sigmoids = compute_forward(z2, output_layer)

    loss = binary_cross_entropy_loss(output_sigmoids, y)


    # l2_gradients = compute_backpropogation(output_sigmoids, y, sigmoids2)


    # l1_gradients = compute_backpropogation(sigmoids2, y, x)

    # output_layer1 = step(hidden_layer1, l1_gradients, 0.1)

    gradients = compute_backpropogation(sigmoids = [sigmoids2, output_sigmoids], layers = [hidden_layer1, output_layer], y=y, x=x)
    hidden_layer1, output_layer1 = step(weights = [hidden_layer1, output_layer], gradients = gradients, lr = 0.001)
    # print(gradients)
    

    print(f"----------------------epoch: {epoch}-----------------------")
    print(f"predictions:\n{(output_sigmoids > 0.5).astype(int)}\nActual:\n{y}, \n {output_sigmoids}")
    print(f"error: {loss}")
    

    # update_all_layers([hidden_layer1, output_layer])

[[0.37547358 0.50973342 0.25291365]
 [0.85255273 0.26147232 0.95869677]] [[0.75498399 0.16564308 0.35487226]
 [0.96016676 0.5314894  0.96910551]]
----------------------epoch: 0-----------------------
predictions:
[[1 1]
 [1 1]
 [1 1]]
Actual:
[[0 0]
 [0 1]
 [0 0]], 
 [[0.77098296 0.90474293]
 [0.81435792 0.95230385]
 [0.81435792 0.95230385]]
error: 5.736748502010941
----------------------epoch: 1-----------------------
predictions:
[[1 1]
 [1 1]
 [1 1]]
Actual:
[[0 0]
 [0 1]
 [0 0]], 
 [[0.77094424 0.90468893]
 [0.81431488 0.95226689]
 [0.81431488 0.95226689]]
error: 5.736950225164719
----------------------epoch: 2-----------------------
predictions:
[[1 1]
 [1 1]
 [1 1]]
Actual:
[[0 0]
 [0 1]
 [0 0]], 
 [[0.7709055  0.9046349 ]
 [0.81427181 0.95222989]
 [0.81427181 0.95222989]]
error: 5.737152053950748
----------------------epoch: 3-----------------------
predictions:
[[1 1]
 [1 1]
 [1 1]]
Actual:
[[0 0]
 [0 1]
 [0 0]], 
 [[0.77086675 0.90458083]
 [0.81422874 0.95219285]
 [0.81422874 

In [ ]:
def forward(inputs):
    inputs = in

array([[0.13692008, 0.4647963 ],
       [0.65395656, 0.00602726]])

In [76]:
import torch
import torch.nn as nn
import torch.optim as optim

In [376]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()


        # self.input = nn.Linear(2,2)
        self.hidden_layer1 = nn.Linear(2,1)
        self.output_layer = nn.Linear(1,2)

    def forward(self, x):
        # x = self.input(x)
        x = torch.sigmoid(x)
        x = self.hidden_layer1(x)
        x = torch.sigmoid(x)
        x = self.output_layer(x)

        return x
    
model = ANN()

x_vec = torch.tensor( [0.1,0.2] )
target = torch.tensor([1.0, 2])

# ----- Forward Propagation -----
output = model(x_vec)

# ----- Compute Loss -----
criterion = nn.BCEWithLogitsLoss()
loss = criterion(output, target)

# ----- Backpropagation -----
optimizer = optim.SGD(model.parameters(), lr = 0.01)
optimizer.zero_grad() ## remove old gradients
loss.backward()
optimizer.step()



